In [ ]:
from pathlib import Path
BASE_DIR = Path(BASE_DIR)  # ensures you can do BASE_DIR / 'file.pkl'

In [ ]:
MODEL_NAME = 'hybrid'

In [ ]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import zipfile
from pathlib import Path

In [ ]:
zip_path = '/content/cnn_data.pkl.zip'
extract_to = '/content/processed_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_to)

print("Unzipped to:", extract_to)

FileNotFoundError: [Errno 2] No such file or directory: '/content/cnn_data.pkl.zip'

In [ ]:

BASE_DIR = Path('/content')

# Sequence length  (match your LSTM window size)
WINDOW_LEN = 50

# Pitch range for piano-roll
PITCH_START = 21
PITCH_END   = 108
NUM_PITCHES = PITCH_END - PITCH_START + 1  # 88

### Define Piano-Roll Conversion

In [ ]:
def seq_to_pianoroll(seq,
                     pitch_start=PITCH_START,
                     num_pitches=NUM_PITCHES):
    """
    seq: np.array of shape (WINDOW_LEN, 3), where seq[:,0] = pitch.
    Returns a binary piano-roll: shape (1, num_pitches, WINDOW_LEN).
    """
    pr = np.zeros((num_pitches, seq.shape[0]), dtype=np.float32)
    for t, note in enumerate(seq):
        p = int(note[0])
        pr[p - pitch_start, t] = 1.0
    return pr[np.newaxis, :, :]

### Load LSTM Windows & Build CNN Features

In [ ]:
import pickle, numpy as np
from collections import Counter

with open(BASE_DIR/'lstm_data.pkl','rb') as f:
    data = pickle.load(f)

# Handle either tuple/list or dict-shaped pickles
if isinstance(data, dict):
    # try common key names; adjust if your dict uses different ones
    X_lstm = data.get('X') or data.get('X_lstm') or data.get('X_windows')
    y      = data.get('y') or data.get('labels')
    le     = data.get('le') or data.get('label_encoder')
else:
    X_lstm, y, le = data  # as in your code

# Basic sanity prints
print("Type(X_lstm):", type(X_lstm))
try:
    print("len(X_lstm):", len(X_lstm))
except TypeError:
    print("X_lstm has no len(); shape:", getattr(X_lstm, 'shape', 'N/A'))

print("Type(y):", type(y), "len(y):", len(y) if hasattr(y, '__len__') else 'N/A')
if hasattr(le, 'classes_'):
    print("Label classes in encoder:", le.classes_)

# Peek at labels
try:
    print("Label sample:", list(y)[:5])
except Exception as e:
    print("Could not preview labels:", e)

# Check class distribution (works if y is list/array of strings/ints)
try:
    print("Class counts:", Counter(list(y)))
except Exception as e:
    print("Could not count classes:", e)

assert hasattr(X_lstm, '__len__') and len(X_lstm) == len(y), "X and y must be same length"
assert len(X_lstm) > 0, "X_lstm is empty — upstream data build likely failed."


In [ ]:
# --- Unified Data Loading & Conversion (tuple/dict-safe) ---
from pathlib import Path
import pickle, numpy as np

def _pick_first(d, keys):
    for k in keys:
        if k in d and d[k] is not None:
            return d[k]
    return None

def load_splits(base: Path):
    # Optional label encoder
    le = None
    le_path = base / 'label_encoder.pkl'
    if le_path.exists():
        with open(le_path, 'rb') as f:
            le = pickle.load(f)

    # Train split can be tuple OR dict depending on your preprocessing run
    with open(base / 'lstm_data.pkl', 'rb') as f:
        data = pickle.load(f)

    if isinstance(data, dict):
        X_train_l = _pick_first(data, ['X','X_lstm','X_windows'])
        y_train   = _pick_first(data, ['y','labels'])
        le_in     = _pick_first(data, ['le','label_encoder'])
        if le is None: le = le_in
    else:
        # Expect (X_train_l, y_train, le) or (X_train_l, y_train)
        if len(data) == 3:
            X_train_l, y_train, le_in = data
            if le is None: le = le_in
        elif len(data) == 2:
            X_train_l, y_train = data
        else:
            raise ValueError(f"Unexpected lstm_data.pkl format: len={len(data)}")

    # Dev/Test are expected as tuples
    with open(base / 'lstm_dev.pkl', 'rb') as f:
        X_dev_l, y_dev = pickle.load(f)
    with open(base / 'lstm_test.pkl', 'rb') as f:
        X_test_l, y_test = pickle.load(f)

    if hasattr(le, "classes_"):
        print("Label classes:", le.classes_)
    else:
        print("[warn] Label encoder missing or has no classes_")

    return X_train_l, y_train, X_dev_l, y_dev, X_test_l, y_test, le

def to_pianoroll_batch(seqs):
    # assumes seq_to_pianoroll(seq) is defined earlier in the notebook
    return np.stack([seq_to_pianoroll(s) for s in seqs], axis=0)

# BASE_DIR must be set (or injected by the orchestrator) to your processed_data folder
X_train_l, y_train, X_dev_l, y_dev, X_test_l, y_test, le = load_splits(BASE_DIR)

X_train_c = to_pianoroll_batch(X_train_l)
X_dev_c   = to_pianoroll_batch(X_dev_l)
X_test_c  = to_pianoroll_batch(X_test_l)

print("LSTM shapes:", getattr(X_train_l, 'shape', len(X_train_l)), getattr(X_dev_l, 'shape', len(X_dev_l)), getattr(X_test_l, 'shape', len(X_test_l)))
print("CNN shapes: ", X_train_c.shape, X_dev_c.shape, X_test_c.shape)
print("Labels train/dev/test:", len(y_train), len(y_dev), len(y_test))


In [ ]:

np.save(BASE_DIR/'cnn_train_data.npy',  X_train_c)
np.save(BASE_DIR/'cnn_train_labels.npy', y_train)
np.save(BASE_DIR/'cnn_dev_data.npy',    X_dev_c)
np.save(BASE_DIR/'cnn_dev_labels.npy',   y_dev)
np.save(BASE_DIR/'cnn_test_data.npy',   X_test_c)
np.save(BASE_DIR/'cnn_test_labels.npy',  y_test)
print("✅ CNN feature files written.")

### Create dataloaders

In [ ]:
# Hybrid Dataset & DataLoaders
import torch
from torch.utils.data import Dataset, DataLoader

class HybridDataset(Dataset):
    def __init__(self, lstm_data, cnn_data, labels):
        self.lstm = torch.tensor(lstm_data, dtype=torch.float32)
        self.cnn  = torch.tensor(cnn_data,  dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.lstm[idx], self.cnn[idx], self.labels[idx]

# Build datasets
train_ds = HybridDataset(X_train_l, X_train_c, y_train)
dev_ds   = HybridDataset(X_dev_l,   X_dev_c,   y_dev)
test_ds  = HybridDataset(X_test_l,  X_test_c,  y_test)

# Create loaders
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
dev_loader   = DataLoader(dev_ds,   batch_size=batch_size)
test_loader  = DataLoader(test_ds,  batch_size=batch_size)


lstm_b, cnn_b, y_b = next(iter(train_loader))
print("LSTM batch shape:", lstm_b.shape)
print("CNN batch shape: ", cnn_b.shape)
print("Label batch shape:", y_b.shape)

### Defining a Hybrid Model

In [ ]:
# Device selection
device = torch.device('mps' if torch.backends.mps.is_available()
                      else 'cuda' if torch.cuda.is_available()
                      else 'cpu')
print("Using device:", device)

class HybridNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # CNN branch
        self.cnn_branch = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(16,32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )
        # LSTM branch
        self.lstm_branch = nn.LSTM(
            input_size=3,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )
        # classifier
        self.fc = nn.Linear(32 + 64, num_classes)

    def forward(self, x_lstm, x_cnn):
        # LSTM path
        out_l, _ = self.lstm_branch(x_lstm)
        feat_l   = out_l[:, -1, :]
        # CNN path
        feat_c   = self.cnn_branch(x_cnn)
        feat_c   = feat_c.view(feat_c.size(0), -1)
        # Concatenate and classify
        combined = torch.cat([feat_l, feat_c], dim=1)
        return self.fc(combined)

# Instantiate
num_classes = len(le.classes_)
model = HybridNet(num_classes).to(device)
print(model)

### Training Setup

In [ ]:
# Hyperparameters
batch_size    = 32
learning_rate = 1e-3
num_epochs    = 20

# Loss and optimizer
criterion     = nn.CrossEntropyLoss()
optimizer     = optim.Adam(model.parameters(), lr=learning_rate)

# Track best dev accuracy
best_dev_acc  = 0.0

print(f"Training for {num_epochs} epochs on device {device}")

### Hybrid Model Training Loop

In [ ]:
for epoch in range(1, num_epochs + 1):
    # ——— Training ———
    model.train()
    train_losses = []
    for x_lstm, x_cnn, yb in train_loader:
        x_lstm, x_cnn, yb = x_lstm.to(device), x_cnn.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(x_lstm, x_cnn)
        loss   = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    avg_train_loss = np.mean(train_losses)

    # ——— Validation ———
    model.eval()
    val_preds, val_true, val_losses = [], [], []
    with torch.no_grad():
        for x_lstm, x_cnn, yb in dev_loader:
            x_lstm, x_cnn, yb = x_lstm.to(device), x_cnn.to(device), yb.to(device)
            logits = model(x_lstm, x_cnn)
            loss   = criterion(logits, yb)
            val_losses.append(loss.item())
            preds = logits.argmax(dim=1).cpu().numpy()
            val_preds.extend(preds)
            val_true.extend(yb.cpu().numpy())

    avg_val_loss = np.mean(val_losses)
    val_acc      = accuracy_score(val_true, val_preds)

    print(f"Epoch {epoch:2d}/{num_epochs}  "
          f"Train Loss: {avg_train_loss:.4f}  "
          f"Val Loss: {avg_val_loss:.4f}  "
          f"Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_dev_acc:
        best_dev_acc = val_acc
        torch.save(model.state_dict(), BASE_DIR / 'best_hybrid.pth')
        print("  🔖 New best hybrid model saved")

In [ ]:

import json, os
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

def _safe_metrics_dump(model_name, y_true_names, y_pred_names, labels):
    acc = accuracy_score(y_true_names, y_pred_names)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true_names, y_pred_names, average='macro', zero_division=0)
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_true_names, y_pred_names, average='weighted', zero_division=0)
    report = classification_report(y_true_names, y_pred_names, digits=4, output_dict=True)
    outdir = Path('/mnt/data/artifacts')
    outdir.mkdir(parents=True, exist_ok=True)
    with open(outdir / f'metrics_{model_name}.json', 'w') as f:
        json.dump({
            "model": model_name,
            "accuracy": acc,
            "precision_macro": precision_macro,
            "recall_macro": recall_macro,
            "f1_macro": f1_macro,
            "precision_weighted": precision_weighted,
            "recall_weighted": recall_weighted,
            "f1_weighted": f1_weighted,
            "classification_report": report,
            "labels": list(labels)
        }, f, indent=2)
    cm = confusion_matrix(y_true_names, y_pred_names, labels=labels)
    plt.figure()
    im = plt.imshow(cm)
    plt.colorbar(im)
    plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
    plt.yticks(range(len(labels)), labels)
    for (i, j), val in np.ndenumerate(cm):
        plt.text(j, i, int(val), ha='center', va='center')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.tight_layout()
    plt.savefig(outdir / f'confusion_{model_name}.png', dpi=150)
    plt.close()

try:
    _labels = le.classes_ if 'le' in globals() else sorted(set(y_true_names) | set(y_pred_names))
    _model_name = MODEL_NAME if 'MODEL_NAME' in globals() else 'model'
    _safe_metrics_dump(_model_name, y_true_names, y_pred_names, _labels)
    print(f"[patch] Saved metrics and confusion matrix for {_model_name} to /mnt/data/artifacts")
except Exception as e:
    print("[patch] Unable to compute metrics from this notebook:", e)
